In [2]:
import h5py
from PIL import Image
import cv2
import numpy as np
import math


In [5]:
#EDIT
filename = "C:/users/brend/Desktop/project3/MNIST_synthetic.h5"

params = cv2.SimpleBlobDetector_Params()
params.minThreshold = 1
params.maxThreshold = 255
params.filterByArea = True
params.minArea = 5
params.maxArea = 130
params.filterByCircularity = False
params.filterByConvexity = False
params.filterByInertia = False
params.blobColor = 255


detector = cv2.SimpleBlobDetector_create(params)

f = h5py.File(filename,'r')
print (f)
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])
print (data[0].ravel())
# retval, threshold = cv2.threshold(data[1], 1, 255, cv2.THRESH_BINARY_INV)

keypoints = detector.detect(data[1])
im = Image.fromarray(data[1][:,:,0])
#EDIT
im.save("C:/users/brend/Desktop/project3/output.jpeg")
im.show()
print (keypoints)

for k in keypoints:
    x = math.floor(k.pt[0])
    y = math.floor(k.pt[1])

    d = data[1][y-7:y+7, x-7:x+7]
    print (d)

    im = Image.fromarray(d[:,:,0])
    #EDIT
    im.save("C:/users/brend/Desktop/project3/output.jpeg")
    im.show()

# im_with_keypoints = cv2.drawKeypoints(threshold, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# # Show keypoints
# cv2.imshow("Keypoints", im_with_keypoints)
# cv2.waitKey(0)



f.close()

<HDF5 file "MNIST_synthetic.h5" (mode r)>
Keys: <KeysViewHDF5 ['test_dataset', 'train_dataset', 'train_labels']>
[0 0 0 ... 0 0 0]
[<KeyPoint 00000261801887B0>, <KeyPoint 00000261FCD38DB0>, <KeyPoint 00000261FCD3B8A0>]
[[[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  1]
  [  1]
  [  1]
  [  0]
  [  0]
  [  0]
  [  0]]

 [[  1]
  [  0]
  [  0]
  [  0]
  [  0]
  [ 20]
  [ 95]
  [174]
  [218]
  [202]
  [ 81]
  [  4]
  [  0]
  [  0]]

 [[ 11]
  [  0]
  [  0]
  [  0]
  [ 28]
  [176]
  [249]
  [251]
  [251]
  [251]
  [186]
  [ 11]
  [  0]
  [  0]]

 [[  1]
  [  0]
  [  0]
  [  0]
  [ 71]
  [214]
  [203]
  [186]
  [249]
  [251]
  [127]
  [  2]
 

In [ ]:
### POSSIBLY HELPFUL CODE


def threshold_image_list(img_list, threshold, binary=False, show_images=False):
    img_thresh = []
    if binary:
        thresh_method = cv2.THRESH_BINARY
    else :
        thresh_method = cv2.THRESH_TOZERO
    for i in range(len(img_list)):
        ret, img = cv2.threshold(img_list[i], threshold, 255, thresh_method)
        img_thresh.append(img)
        if show_images:
            plt.imshow(img, cmap='gray', vmin=0, vmax=255)
            plt.show()

    return img_thresh


def detect_blob_locations(img_list, threshold, show_images=False):
    modified = False

    threshold = 255

    params = cv2.SimpleBlobDetector_Params()

    params.minThreshold = threshold
    params.maxThreshold = 255

    params.filterByArea = True
    params.minArea = 20
    params.maxArea = 200

    params.filterByCircularity = False
    params.filterByInertia = False
    params.filterByConvexity = False
    params.filterByColor = False
    params.blobColor = 255

    detector = cv2.SimpleBlobDetector_create(params)

    keypoints_list = []
    for img in img_list:
        img = img.astype(np.uint8)
        keypoints = detector.detect(img)

        while(len(keypoints) < 3 and params.minThreshold > 100 ):
            modified = True
            params.minThreshold = params.minThreshold - 1
            # print(params.minThreshold)
            detector = cv2.SimpleBlobDetector_create(params)
            keypoints = detector.detect(img)
            # print(len(keypoints))

            # img_with_keypoints = cv2.drawKeypoints(img, keypoints[:], np.array([]), (0, 0, 255),
            #                                        cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            # # plt.imshow(img_with_keypoints, cmap='gray', vmin=0, vmax=255)
            # # plt.show()

        keypoints_list.append(keypoints)

        if modified:
            modified = False
            #reset parameters
            params.minThreshold = threshold
            detector = cv2.SimpleBlobDetector_create(params)

        if show_images:
            img_with_keypoints = cv2.drawKeypoints(img, keypoints_list[-1], np.array([]), (0, 0, 255),
                                                   cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            plt.imshow(img_with_keypoints, cmap='gray', vmin=0, vmax=255)
            plt.show()

    return keypoints_list